# Check what you are voting for with PyTezos

In [1]:
import sys
assert sys.version_info.major == 3 and sys.version_info.minor >= 6

In [2]:
import os
sys.path.insert(1, os.path.abspath('..'))

## Current voting phase

Say, you don't read any news and all you have is public node access and documentation.  
How to determine what is the current voting phase?  

In [3]:
from pytezos.rpc import mainnet

In [4]:
mainnet.head.votes.current_period_kind()

'proposal'

Great, we are at the first stage, now we want to know when did it start and when it will ends.  
We can get this information from the block metadata:

In [5]:
level_info = mainnet.head.metadata.get('level')
level_info

{'level': 349243,
 'level_position': 349242,
 'cycle': 85,
 'cycle_position': 1082,
 'voting_period': 10,
 'voting_period_position': 21562,
 'expected_commitment': False}

Get precise boundaries in levels and rough estimation in days

In [6]:
import pendulum

In [7]:
start_level = level_info['level'] - level_info['voting_period_position']
start_level

327681

In [9]:
end_level = start_level + 8 * 4096 - 1  # eight cycles of 4096 blocks
end_level

360448

In [10]:
start_dt = pendulum.parse(mainnet.blocks[start_level].header.get('timestamp'))
time_past = (pendulum.now() - start_dt)
print(round(time_past.total_days(), 1), 'days passed')

15.6 days passed


In [11]:
time_left = (end_level - level_info['level']) / (level_info['level'] - start_level) * time_past
print(round(time_left.total_days(), 1), 'days left')

8.1 days left


## Current proposals

In [12]:
proposals = mainnet.head.votes.proposals()
proposals

[['Psd1ynUBhMZAeajwcZJAeq5NrxorM6UCU4GJqxZ7Bx2e9vUWB6z', 5176],
 ['Pt24m4xiPbLDhVgVfABUjirbmda3yohdN82Sp9FeuAXJ4eV9otd', 13183]]

Let's examine one of proposals

In [13]:
proposal_id = proposals[1][0]
proposal_id

'Pt24m4xiPbLDhVgVfABUjirbmda3yohdN82Sp9FeuAXJ4eV9otd'

### Injection operation

It's interesting to find the author of the proposal.  
In order to do that we have to search for the first `proposals` operation for this particular proposal_id.

In [14]:
from pytezos.rpc.search import SearchChain

In [15]:
sc = SearchChain.from_chain(mainnet.main)

Thanks to the statefullness of the Tezos blockchain we can perform a binary search inside the voting period.  
The algorithm searches for the first level where number of votes changed from 0 to non-zero.

In [16]:
operation = sc.find_proposal_inject_operation(proposal_id)

2019-03-12 16:21:03.348 | DEBUG    | pytezos.rpc.search:bisect:29 - 714 at level 338464
2019-03-12 16:21:04.196 | DEBUG    | pytezos.rpc.search:bisect:29 - 23 at level 333072
2019-03-12 16:21:04.985 | DEBUG    | pytezos.rpc.search:bisect:29 - 0 at level 330376
2019-03-12 16:21:05.763 | DEBUG    | pytezos.rpc.search:bisect:29 - 0 at level 331724
2019-03-12 16:21:06.585 | DEBUG    | pytezos.rpc.search:bisect:29 - 0 at level 332398
2019-03-12 16:21:07.351 | DEBUG    | pytezos.rpc.search:bisect:29 - 11 at level 332735
2019-03-12 16:21:07.957 | DEBUG    | pytezos.rpc.search:bisect:29 - 0 at level 332566
2019-03-12 16:21:08.570 | DEBUG    | pytezos.rpc.search:bisect:29 - 11 at level 332650
2019-03-12 16:21:09.646 | DEBUG    | pytezos.rpc.search:bisect:29 - 0 at level 332608
2019-03-12 16:21:10.210 | DEBUG    | pytezos.rpc.search:bisect:29 - 11 at level 332629
2019-03-12 16:21:10.786 | DEBUG    | pytezos.rpc.search:bisect:29 - 0 at level 332618
2019-03-12 16:21:11.661 | DEBUG    | pytezos.rpc

In [17]:
operation()

{'protocol': 'PsddFKi32cMJ2qPjf43Qv5GDWLDPZb3T3bF6fLKiF5HtvHNU7aP',
 'chain_id': 'NetXdQprcVkpaWU',
 'hash': 'onydFJLWdGhfKNBfbnSLmqDu93j9NRimkbQm9WqLWYG8eyZUyTF',
 'branch': 'BL53WJx6xPn6rnTnWZmpNaWGAQqU8HTwRTDqVDmthYsxUTBewo9',
 'contents': [{'kind': 'proposals',
   'source': 'tz1fNdh4YftsUasbB1BWBpqDmr4sFZaPNZVL',
   'period': 10,
   'proposals': ['Pt24m4xiPbLDhVgVfABUjirbmda3yohdN82Sp9FeuAXJ4eV9otd',
    'Psd1ynUBhMZAeajwcZJAeq5NrxorM6UCU4GJqxZ7Bx2e9vUWB6z'],
   'metadata': {}}],
 'signature': 'sigvUqvh7rBS8yAoE5RMieQaD5hvg9NsLeJ4kTnXdK1tXXyrHL8mX7E3KCm9q9YgYbJn3edhcUiZjdU3xNhVPEUPkSGVNbi9'}

We can perform some checks on this operation, such as signature validation:

In [18]:
operation.verify_signature()

The only thing we can learn about the submitter's identity is his public key:

In [19]:
mainnet.get_public_key(operation.source())

'edpktyybTTrLm2rk6bn7xtgY9t2Kgt9GnqZxEcSrunm4vKWTF9ES9o'

In [20]:
timestamp = mainnet.blocks[operation.get('branch')].header.get('timestamp')
inject_dt = pendulum.parse(timestamp)
f'Proposed on the {(inject_dt - start_dt).days + 1}th day of the voting period'

'Proposed on the 4th day of the voting period'

### Source code

Unfortunately it's practically impossible to get proposal source from the blockchain.  
We could get lucky and find the submitter's node (in case it's public) which has to know this proto (according to the docs).  
The other option is trying to find a node in zeronet which has participated in the voting rehearsal.  

We will be back to this issue later, now let's download proposal sources.

In [21]:
tar_url = 'https://blog.nomadic-labs.com/files/Athens_proposal_A.tar'

In [22]:
from pytezos.rpc.protocol import Protocol

Loading sources and convert them to the internal format (as in blockchain)

In [23]:
athens_a = Protocol.from_uri(tar_url)

1218560it [03:36, 5620.72it/s]


Check that the sources we have downloaded are original.  
In order to do that we have to obtain binary representation of the sources according to the http://tezos.gitlab.io/mainnet/api/rpc.html#get-protocols-protocol-hash (Binary output tab).

In [24]:
proto_id = athens_a.calculate_hash()
proto_id

'Pt24m4xiPbLDhVgVfABUjirbmda3yohdN82Sp9FeuAXJ4eV9otd'

In [25]:
assert proposal_id == proto_id

### Protocol update diff

First of all we need to get sources of the current protocol:

In [26]:
current_proto = mainnet.protocols[operation.get('protocol')]
current_proto

protocols/PsddFKi32cMJ2qPjf43Qv5GDWLDPZb3T3bF6fLKiF5HtvHNU7aP

In [27]:
current_hash = current_proto.calculate_hash()
current_hash

'PsddFKi32cMJ2qPjf43Qv5GDWLDPZb3T3bF6fLKiF5HtvHNU7aP'

In [28]:
assert current_hash == operation.get('protocol')

Now we can generate patch file in the standard diff format.  
We can optional specify number of lines before and after the change: this is useful for review.

In [29]:
patch = current_proto.diff(athens_a, context_size=3)

Generate github-like side-by-side patch views, powered by diff2html.js

In [30]:
patch.export_html(output_path='report.html')

[View report](./report.html)

## Bonus: get all voting operations for a proposal

As an alternative to TzScan and other indexed-blockchain solutions.

In [31]:
for operation in sc.find_proposal_votes_operations(proposal_id):
    print(operation())
    break  # this can take a while

2019-03-12 16:27:29.640 | DEBUG    | pytezos.rpc.search:find_state_change_intervals:14 - 13179 at level 349194
2019-03-12 16:27:30.438 | DEBUG    | pytezos.rpc.search:bisect:29 - 13183 at level 349224
2019-03-12 16:27:31.570 | DEBUG    | pytezos.rpc.search:bisect:29 - 13183 at level 349209
2019-03-12 16:27:32.367 | DEBUG    | pytezos.rpc.search:bisect:29 - 13183 at level 349201
2019-03-12 16:27:33.171 | DEBUG    | pytezos.rpc.search:bisect:29 - 13179 at level 349197
2019-03-12 16:27:33.973 | DEBUG    | pytezos.rpc.search:bisect:29 - 13183 at level 349199
2019-03-12 16:27:35.187 | DEBUG    | pytezos.rpc.search:bisect:29 - 13179 at level 349198


{'protocol': 'PsddFKi32cMJ2qPjf43Qv5GDWLDPZb3T3bF6fLKiF5HtvHNU7aP', 'chain_id': 'NetXdQprcVkpaWU', 'hash': 'oomv4x6eVaa2EbT37h7MvCZEMyKBiqf3L4yuEoFE95n9aKYfGxF', 'branch': 'BLDU8tQwnZEt3So7Utbwxzj7WMLQdyNqu7UkzuuAJKUnnNmenHK', 'contents': [{'kind': 'proposals', 'source': 'tz1Z3KCf8CLGAYfvVWPEr562jDDyWkwNF7sT', 'period': 10, 'proposals': ['Pt24m4xiPbLDhVgVfABUjirbmda3yohdN82Sp9FeuAXJ4eV9otd'], 'metadata': {}}], 'signature': 'sigqAsAk9wWFSz5zDt3nn3M5yf14ynxNSw23MAwDmwv5kifGtNyqDbi1aoxiAAF5byyzqjgu3ftWYxc8DHKsB6cGWr1nHh5S'}


Search algorithm works as follows:
1. Split block interval into equal chunks
2. Determine which of the intervals contain state changes
3. For each interval run binary search
4. If there are several changes inside single interval, run binary search again

It's obvious that the search space can be easily splitted and processed independently, i.e parallelized. 

## Improvement: proposal sources on-chain

As we pointed earlier there is no convenient way to get proposal source from the blockchain. This can be implemented via smart-contract. But it's more reasonable to store compressed code diff rather than full source.

In [32]:
ctxless_patch = current_proto.diff(athens_a, context_size=0)

In [33]:
ctxless_patch.export_tar('diff.tar.bz2')

In [34]:
os.path.getsize('diff.tar.bz2')

12839

### Applying protocol diff

In [35]:
proto = current_proto.apply(ctxless_patch)

In [36]:
assert proposal_id == proto.calculate_hash()